In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

from itertools import cycle
from matplotlib import pyplot as plt

from simulation import SimulationMixin


In [2]:
fontsize = 20

plots_dir = "plots"

In [3]:
colors = [("red", "red"), ("green", "green"), ("blue", "blue"), ("orange", "orange")]
color_cycle = cycle(colors)

In [ ]:
program = "coloring"
# program = "maximal_matching"
# program = "dijkstra"
# graph_name = "graph_powerlaw_cluster_graph_n7"
# graph_name = "implicit_graph_n8"
graph_name = "graph_10"
sched = 0
no_simulations = 10000
me = False
fault_interval = 2

duong_mode = True

selected_nodes = [0, 4, 7, 9]

In [5]:
def get_filename(graph_name, sched, simulation_type, args, no_simulations, me, fault_interval):
    return f"{graph_name}__{sched}__{simulation_type}_args_{args}__{no_simulations}__{me}__{fault_interval}"

In [6]:
filenames = [
    get_filename(
        graph_name,
        sched,
        SimulationMixin.RANDOM_FAULT_SIMULATION_TYPE,
        "",
        no_simulations,
        me,
        fault_interval,
    )
]
if duong_mode:
    filenames.extend(
        [
            get_filename(
                graph_name,
                sched,
                SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE_DUONG,
                arg,
                no_simulations,
                me,
                fault_interval,
            )
            for arg in selected_nodes
        ]
    )
else:
    filenames.extend(
        [
            get_filename(
                graph_name,
                sched,
                SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE,
                arg,
                no_simulations,
                me,
                fault_interval,
            )
            for arg in selected_nodes
        ]
    )


In [7]:
dfs = [pd.read_csv(os.path.join("results", program, f"{fn}.csv")) for fn in filenames]

In [8]:
max_steps = max(df['Steps'].max() for df in dfs)
max_steps

np.int64(37)

In [9]:
bins = np.linspace(0, max_steps, max_steps+1)
# bin_centers = (bins[:-1] + bins[1:]) / 2
bin_centers = bins

In [10]:
bins

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37.])

In [11]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_data[0][0]

array([   0, 4038,  559, 2285,  308, 1195,  179,  628,   92,  306,   49,
        156,   19,   75,   18,   38,    6,   20,    2,   12,    3,    5,
          0,    2,    0,    3,    0,    1,    0,    0,    0,    0,    0,
          1,    0,    0,    0])

In [12]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_df = []
for hd in hist_data:
    hist_df.append(
        pd.DataFrame({
            'Steps': bin_centers[:-1],
            'Count': hd[0]
        })
    )

In [13]:
hist_df[0]

,Steps,Count
0,0.0,0
1,1.0,4038
2,2.0,559
3,3.0,2285
4,4.0,308
5,5.0,1195
6,6.0,179
7,7.0,628
8,8.0,92
9,9.0,306


In [14]:
df_merged = hist_df[0]
for i in range(1, len(hist_df)):
    df_merged = pd.merge(df_merged, hist_df[i], on=["Steps"], suffixes=(i-1, i))
df_merged

,Steps,Count0,Count1,Count2,Count3,Count
0,0.0,0,0,0,0,0
1,1.0,4038,3994,4067,3939,4033
2,2.0,559,449,593,428,600
3,3.0,2285,2382,2292,2429,2244
4,4.0,308,326,335,259,354
5,5.0,1195,1218,1225,1299,1184
6,6.0,179,132,164,125,183
7,7.0,628,660,601,671,609
8,8.0,92,90,88,67,94
9,9.0,306,326,261,356,289


In [15]:
df_merged.set_index('Steps', drop=True, inplace=True)
df_merged

,Count0,Count1,Count2,Count3,Count
Steps,,,,,
0.0,0,0,0,0,0
1.0,4038,3994,4067,3939,4033
2.0,559,449,593,428,600
3.0,2285,2382,2292,2429,2244
4.0,308,326,335,259,354
5.0,1195,1218,1225,1299,1184
6.0,179,132,164,125,183
7.0,628,660,601,671,609
8.0,92,90,88,67,94


In [16]:
def get_title():
    return f"Simulation - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"

In [17]:
def get_filename():
    return f"{program}__{graph_name}__{sched}__{no_simulations}__{fault_interval}__{''.join([str(i) for i in selected_nodes])}{'__duong' if duong_mode else ''}"

In [18]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, linewidth=3)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

# ax.set_yscale("log")
for i, line in enumerate(ax.lines):
    if i >= 1:
        line_style = "solid"
        line.set_color(next(color_cycle)[0])
    else:
        line_style = "dashed"
        line.set_color("goldenrod")
    line.set_linestyle(line_style)

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)

ax.set_xlabel("Steps")
ax.set_ylabel("Count")

labels = ["Random Fault"]
labels.extend(
    [
        f'Controlled {"(duong)" if duong_mode else ""} at node {n}'
        for n in selected_nodes
    ]
)
custom_lines = [
    mlines.Line2D(
        [],
        [],
        color=line.get_color(),
        # marker=marker,
        label=cat,
        linewidth=1,
        linestyle=line.get_linestyle(),
    )
    for line, cat in zip(ax.lines, labels)
]
plt.rc("font", size=fontsize)
plt.legend(handles=custom_lines, fontsize=fontsize * 0.9)
file_path = os.path.join(
    plots_dir,
    program,
    f"{get_filename()}.png",
)
plt.savefig(
    file_path,
    bbox_inches="tight",
)
plt.close()

print(f"Saved plot(s) for {file_path}")

Saved plot(s) for plots/coloring/coloring__graph_7__0__10000__2__0459__duong.png


In [19]:
# Save to file
df_merged.columns = [
    "Random",
    *[
        f'Controlled {"(duong)" if duong_mode else ""} at node {n}'
        for n in selected_nodes
    ],
]
df_merged.index = df_merged.index.astype(int)
df_merged.to_csv(
    os.path.join("results", program, f"agg_{get_filename()}.csv")
)